# Continous motion classification


In this project we will use Machine Learning to detect which gesture the user is performing by means of the accelerometer and gyroscope data coming from an IMU (Inertia Measurement Unit) sensor.

This task differs from the previous ones since it involves *time* as a feature. We are no more working with static data: we'll work with values that change over time.

We will learn that, when dealing with time series data, we need to use a *windowing scheme*, to groups sensor readings in chunks to be processed. From each chunk we can later extract a set of *spectral features* to use as inputs for classification.

This project is a replica of [the one from Edge Impulse](https://docs.edgeimpulse.com/docs/tutorials/continuous-motion-recognition). Instead of using Neural Networks, though, we will use a "traditional" classifier.

## Hardware requirements

To follow this project you need an IMU sensor.

It can either be built-in your board (e.g Arduino BLE Sense) or an external sensor (e.g. MPU9250).

## Software requirements

To implement the Machine Learning part in Python you will need the [`everywhereml`](https://github.com/eloquentarduino/everywhereml) library.

In [3]:
# !pip install everywhereml>=0.0.3

## Data collection

We will collect training data via the Serial port of your PC.

If you followed the [Fruit classification](#) project, the next piece of code will look familiar.

First of all, create a sketch that is able to print accelerometer and gyroscope data. This will change based on your board and sensor; here is an example for the Arduino BLE Sense.

```cpp
// file IMUCollect.ino

#include <Arduino_LSM9DS1.h>


void setup() {
  Serial.begin(115200);
  while (!Serial);
  Serial.println("Started");

  if (!IMU.begin()) {
    Serial.println("Failed to initialize IMU!");
    while (1);
  }

  Serial.print("Accelerometer sample rate = ");
  Serial.print(IMU.accelerationSampleRate());
  Serial.println("Hz");
}

void loop() {
  float ax, ay, az, gx, gy, gz;
  
  if (!IMU.accelerationAvailable() || !IMU.gyroscopeAvailable())
    return;
    
  IMU.readAcceleration(ax, ay, az);
  IMU.readGyroscope(gx, gy, gz);

  Serial.print("IMU: ");
  Serial.print(ax);
  Serial.print(",");
  Serial.print(ay);
  Serial.print(",");
  Serial.print(az);
  Serial.print(",");
  Serial.print(gx);
  Serial.print(",");
  Serial.print(gy);
  Serial.print(",");
  Serial.print(gz);
  Serial.print("\n");
}
```

Then run the following Python code. It will connect to the Serial port of your board and start reading whatever gets printed. Everytime it finds a well-formed line of data, the program will add it to a list that you can eventually save to a file for later use.

In [4]:
from everywhereml.data import Dataset
from everywhereml.data.collect import SerialCollector

"""
Create a SerialCollector object.
Each data line is marked by the 'IMU:' string
Collect 30 seconds of data for each gesture
Replace the port with your own!

If a imu.csv file already exists, skip collection
"""

try:
    imu_dataset = Dataset.from_csv(
        'imu.csv', 
        name='ContinuousMotion', 
        target_name_column='target_name'
    )
    
except FileNotFoundError:
    imu_collector = SerialCollector(
        port='/dev/cu.usbmodem141401', 
        baud=115200, 
        start_of_frame='IMU:', 
        feature_names=['ax', 'ay', 'az', 'gx', 'gy', 'gz']
    )
    imu_dataset = imu_collector.collect_many_classes(
        dataset_name='ContinuousMotion', 
        duration=30
    )
    
    # save dataset to file for later use
    imu_dataset.df.to_csv('imu.csv', index=False)

EmptyDataError: No columns to parse from file

In [ ]:
"""
Print summary of dataset
"""
imu_dataset.describe()

In [ ]:
"""
Plot features pairplot
Since this is a time series dataset, the pairplot won't be very informative
We will come back to the pairplot after feature pre-processing to see great improvements!
"""
imu_dataset.plot.features_pairplot(n=300)

## Feature extraction

Now that we have collect our dataset, it is time to extract features from it.

When working with time series data, our go-to feature extractor is a combination of `Window` + `Spectral Features`.

`Window` packs a given number of consecutive readings into a single, large array. You can configure it to overlap each window with the previous by a given amount, so as to capture different time slices of the same event.

![Window example](./assets/rolling-window.jpg)

`SpectralFeatures` takes in input the *windowed* data and extracts a number of statistics from it, for example:

 - min / max
 - absolute value min / max
 - mean value
 - variance and standard deviation
 - [skew](https://en.wikipedia.org/wiki/Skewness) and [kurtosis](https://en.wikipedia.org/wiki/Kurtosis)

In [ ]:
from everywhereml.preprocessing import Pipeline, MinMaxScaler, Window, SpectralFeatures

# this is the frequency of your sensor
# change according to your hardware
sampling_frequency = 104
mean_gesture_duration_in_millis = 1000
window_length = sampling_frequency * mean_gesture_duration_in_millis // 1000

imu_pipeline = Pipeline(name='ContinousMotionPipeline', steps=[
    MinMaxScaler(),
    # shift can be an integer (number of samples) or a float (percent)
    Window(length=window_length, shift=0.3),
    # order can either be 1 (first-order features) or 2 (add second-order features)
    SpectralFeatures(order=2)
])

In [ ]:
"""
Enumerate features extracted from the SpectralFeatures step
"""
from pprint import pprint

pprint(imu_pipeline['SpectralFeatures'][0].feature_names)

In [ ]:
"""
Apply feature pre-processing
"""
imu_dataset.apply(imu_pipeline)

In [ ]:
imu_dataset.describe()

In [ ]:
"""
Plot features pairplot after feature extraction
Now it will start to make sense
Since SpectralFeatures generates 8 or 20 features (depending on the order)
for each axis, we limit the visualization to a more reasonable number
"""
imu_dataset.plot.features_pairplot(n=300, k=6)

In [ ]:
"""
Perform classification with a RandomForest
"""
from everywhereml.sklearn.ensemble import RandomForestClassifier

imu_classifier = RandomForestClassifier(n_estimators=20, max_depth=20)
imu_train, imu_test = imu_dataset.split(test_size=0.3)
imu_classifier.fit(imu_train)

print('Score on test set: %.2f' % imu_classifier.score(imu_test))

As you can see from the plot, the `SpectralFeatures` feature extractor can convert the raw data windows to a meaningful set of features that a classifier can use to accurately predict the gesture.

## Port to Arduino

We need to port two pieces of code to Arduino:

 1. the pre-processing pipeline
 2. the classifier
 
Both of them implement the `to_arduino_file()` function to perform this task.

In [ ]:
"""
Port pipeline to C++
"""
print(imu_pipeline.to_arduino_file(
    'sketches/IMUClassify/Pipeline.h', 
    instance_name='pipeline'
))

In [ ]:
"""
Port classifier to C++
"""
print(imu_classifier.to_arduino_file(
    'sketches/IMUClassify/Classifier.h', 
    instance_name='forest', 
    class_map=imu_dataset.class_map
))

With these two pieces in place, it's time to integrate them into a sketch.

```cpp
// file IMUClassify.ino

#include <Arduino_LSM9DS1.h>
#include "Pipeline.h"
#include "Classifier.h"


void setup() {
  Serial.begin(115200);
  Serial.println("Started");

  while (!IMU.begin()) 
    Serial.println("Failed to initialize IMU!");

  Serial.print("Accelerometer sample rate = ");
  Serial.print(IMU.accelerationSampleRate());
  Serial.println("Hz");
}

void loop() {
  float ax, ay, az, gx, gy, gz;
  
  // await for data
  if (!IMU.accelerationAvailable() || !IMU.gyroscopeAvailable())
    return;
    
  IMU.readAcceleration(ax, ay, az);
  IMU.readGyroscope(gx, gy, gz);

  // perform feature extraction
  float features[] = {ax, ay, az, gx, gy, gz};
    
  if (!pipeline.transform(features))
    return;
    
  // perform classification
  Serial.print("Predicted gesture: ");
  Serial.print(forest.predictLabel(pipeline.X));
  Serial.print(" (DSP: ");
  Serial.print(pipeline.latencyInMicros());
  Serial.print(" micros, Classifier: ");
  Serial.print(forest.latencyInMicros());
  Serial.println(" micros)");
}
```

Deploy the sketch to your board and start performing each gesture.

Based on the length of your gestures and the complexity of the classifier, you can expect sub-millisecond DSP time if using first-order features only and 100-200 microseconds for classification.